Analyser.ipynb: 
Uses aggregator to summarise results of phases for quick and easy analysis.

Change "workspace_path" to the location of your workspace, "data_path" to the directory containing the phases you want to analyse and filter results with the "agg.filter" method.

In [ ]:
from autoconf import conf
workspace_path = "/home/matthew/Durham2020/AutoLensWorkspace/autolens_workspace/"
conf.instance = conf.Config(
    config_path=f"{workspace_path}config",
)

In [ ]:
import autolens as al
import autofit as af
import autolens.plot as aplt
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_path="/home/matthew/Durham2020/Outputs/Run3/"
agg = af.Aggregator(directory=data_path)

phase_name = "phase_1"
agg_filter_phase = agg.filter(agg.phase == phase_name)

In [ ]:
def radial_mass_graph(galaxy_dict):
    
    n_directions = 20
    r_array = np.arange(0.0,5,0.01)
    e_radius_list = []
    fig, axs = plt.subplots(1, len(galaxy_dict) + 1, figsize = (25,8))
    axs_index = 0
    warning_bool = False
    def trip_warning(profile):
        return (profile.centre[0]**2 + profile.centre[1]**2 > 0.0**2)
    for galaxy_key in galaxy_dict:
        galaxy = galaxy_dict[galaxy_key]
        e_radius = galaxy.einstein_radius_in_units()
        e_radius_list.append(e_radius)
        conv_array = np.zeros(r_array.shape)
        total_conv_array = np.zeros(r_array.shape)
        angle_array = np.array([np.sin(np.arange(0, 2*np.pi, 2*np.pi/n_directions)),
                                np.cos(np.arange(0, 2*np.pi, 2*np.pi/n_directions))]).transpose()
        N_profiles = 0
        for key in galaxy.__dict__:
            if isinstance(galaxy.__dict__[key], al.mp.MassProfile):
                profile = galaxy.__dict__[key]
                label = key
                N_profiles+=1
                if trip_warning(profile): warning_bool=True
                for i in range(r_array.size):
                    grid = r_array[i]*angle_array
                    conv_array[i] = float(np.average(profile.convergence_from_grid(grid=grid)))
                axs[axs_index].plot(r_array, conv_array, label=label)
                total_conv_array+=conv_array
        if N_profiles>1:
            axs[axs_index].plot(r_array, total_conv_array, label = "Total")
        axs[axs_index].legend()
        axs[axs_index].autoscale(tight=True)
        axs[axs_index].set_yscale('log')
        axs[axs_index].plot([e_radius,e_radius], axs[axs_index].get_ylim(), axs[axs_index].lines[-1].get_color(), linestyle='dashed')
        axs[axs_index].set_xlabel('Distance from centre /arcseconds')
        axs[axs_index].set_ylabel('Average Convergence')
        axs[axs_index].set_title(galaxy_key)
        axs[-1].plot(r_array, total_conv_array, label = galaxy_key)
        axs_index+=1
    
    axs[-1].set_xlabel('Distance from centre /arcseconds')
    axs[-1].set_ylabel('Average Convergence')
    axs[-1].set_title("Totals")
    axs[-1].legend()
    axs[-1].autoscale(tight=True)
    axs[-1].set_yscale('log')
    for index in range(len(e_radius_list)):
        e_radius = e_radius_list[index]
        axs[-1].plot([e_radius,e_radius],axs[-1].get_ylim(),axs[-1].lines[index].get_color(),linestyle='dashed')
    plt.show()
    if warning_bool: print("WARNING: OFF-CENTRE MASS PROFILES")

In [ ]:
for phase in agg:
    output = phase.samples
    dataset = phase.dataset
    mask = phase.mask
    masked_imaging = al.MaskedImaging(imaging=dataset, mask=mask)
    fitted_tracer = al.Tracer.from_galaxies(
        galaxies=output.max_log_likelihood_instance.galaxies
    )
    true_tracer = phase.true_tracer
    
    
    print("\nPhase:")
    if phase.name != None: print(f"\n{phase.name}")
    if dataset.name != None: print(f"\nDataset name: {dataset.name}")
    print(f"\nPhase directory:")
    print(f"{phase.directory}")
    if phase.info != None:
        print("\nInfo:")
        print(f"{phase.info}")
    
    print("\n\nTrue Galaxies:")
    for galaxy in true_tracer.galaxies: print(f"\n{galaxy}")
    
    print("\n\nFitted Galaxies:")
    #for i in range(len(output.parameter_names)):
    #    print(f"{output.parameter_names[i]} = {output.parameters[-1][i]}")
    
    for galaxy in fitted_tracer.galaxies: print(f"\n{galaxy}")
    
    print("\n\nFit Imaging:")
    aplt.FitImaging.subplot_fit_imaging(fit=al.FitImaging(masked_imaging=masked_imaging, tracer=fitted_tracer))
    
    print("\nRadial Mass Distributions: (Dashed lines correspond to Einstein radii of each galaxy)\n")
    radial_mass_graph({
        "True Lens Galaxy": true_tracer.galaxies[0],
        "Fitted Lens Galaxy": fitted_tracer.galaxies[0]
    })
    
    print("\nFitted Source Plane:\n")
    aplt.Plane.plane_image(plane=fitted_tracer.planes[-1], grid=al.Grid.uniform(shape_2d=(100, 100), pixel_scales=0.025))